In [48]:
import pandas as pd
import os
import matplotlib
import itchat
%matplotlib inline

In [51]:
itchat.auto_login()

Getting uuid of QR code.
Please scan the QR code to log in.
Please press confirm on your phone.
Loading the contact, this may take a little while.
Login successfully as 邓卫东


In [54]:
itchat.run()

Start auto replying.
Bye~


In [8]:
user_df=pd.read_csv('./data/t_user.csv')
order_df=pd.read_csv('./data/t_order.csv')
loan_df=pd.read_csv('./data/t_loan.csv')
loan_sum_df=pd.read_csv('./data/t_loan_sum.csv')
click_df=pd.read_csv('./data/t_click.csv')

In [288]:
loan_dt=datetime.datetime.strptime('2016-11-01 00:00:00','%Y-%m-%d %H:%M:%S')
loan_df['loan_gap']=loan_df['loan_time'].apply(lambda x: loan_dt- (datetime.datetime.strptime(x,'%Y-%m-%d %H:%M:%S')))
loan_df['loan_gap']=loan_df['loan_gap'].apply(lambda x: x.days)

### 划分样本集

In [293]:
train_time='2016-11-01'
span_time='2016-09-01'
train_order_df=order_df[order_df['buy_time']<train_time]
train_click_df=click_df[click_df['click_time']<train_time]
train_loan_df=loan_df[loan_df['loan_time']<train_time]
span_order_df=order_df[order_df['buy_time']>=span_time]
span_click_df=click_df[click_df['click_time']>=span_time]
span_loan_df=loan_df[loan_df['loan_time']>=span_time]

In [294]:
train_loan_df

,uid,loan_time,loan_amount,plannum,loan_gap
0,12135,2016-08-03 00:05:26,3.862595,1,89
1,41403,2016-08-03 00:13:25,5.584137,3,89
2,74458,2016-08-03 00:13:58,4.723017,1,89
3,12959,2016-08-03 00:19:33,3.862595,1,89
4,89641,2016-08-03 00:23:13,4.292651,1,89
5,3272,2016-08-03 00:24:32,3.862595,1,89
6,90639,2016-08-03 00:36:19,4.544373,1,89
7,66044,2016-08-03 00:50:20,4.622093,1,89
8,89641,2016-08-03 01:55:12,4.292651,1,89
9,41227,2016-08-03 02:26:52,5.070593,3,89


### 获取用户订单信息

In [259]:
def get_order_feature(df):
    order_group=df.groupby('uid')
    order_feature=[]
    for uid,group in order_group:
        group=group.reset_index()
        order_feature_dict={}
        order_feature_dict['uid']=uid
        order_feature_dict['total_order']=len(group)
        order_feature_dict['unique_cate_count']=len(group['cate_id'].unique())
        order_feature_dict['max_qty']=group['qty'].max()
        order_feature_dict['max_price']=group['price'].max()
        order_feature_dict['max_discount']=group['discount'].max()
        total_price=0
        for i in range(len(group)):
            total_price+=group.loc[i,'qty']*group.loc[i,'price']
        order_feature_dict['total_price']=total_price
        order_feature.append(order_feature_dict)
    order_feature_df=pd.DataFrame(order_feature)
    return order_feature_df

In [175]:
order_feature_df.to_csv('./feature/order_feature.csv',index=False)

### 提取历史借贷行为 

In [252]:
loan_group=loan_df.groupby('uid')

In [310]:
def get_loan_feature(df):
    loan_group=df.groupby('uid')
    loan_feature=[]
    cnt=0
    for uid,group in loan_group:
        cnt+=1
        if cnt%10000==0:
            print "count total user:","cnt"
        loan_feature_dict={}
        group=group.reset_index()
        loan_feature_dict['uid']=uid
        loan_feature_dict['loan_count_3m']=len(group)
        loan_feature_dict['loan_amout__3m']=group['loan_amount'].sum()
        loan_gap=list(group['loan_gap'])
        for day_diff in [7,14,21,28,35,42,49,56,63,70,90]:
            loan_feature_dict['last_'+str(day_diff)+'_day_loan_count']=len([i for i in loan_gap if i <=day_diff])
            loan_feature_dict['last_'+str(day_diff)+'_day_loan_amount']=sum([group.loc[loan_gap.index(i),'loan_amount'] for i in loan_gap if i<=day_diff])
        loan_feature.append(loan_feature_dict)
    return pd.DataFrame(loan_feature)

In [258]:
loan_feature_df=pd.DataFrame(loan_feature)

### merge feature

#### 训练集特征

In [311]:
train_order_feature=get_order_feature(train_order_df)
train_loan_feature=get_loan_feature(train_loan_df)
train_feature=pd.merge(user_df,train_order_feature,on='uid',how='left')
train_feature=pd.merge(train_feature,train_loan_feature,on='uid',how='left')
train_label=pd.merge(user_df,loan_sum_df,on='uid',how='left')
train_label.fillna(0,inplace=True)
train_label=train_label[['uid','loan_sum']]
train_data=pd.merge(train_label,train_feature,on='uid',how='left')
train_data.drop(['active_date'],axis=1,inplace=True)

(155047, 5)
33190
count total user: cnt
count total user: cnt
count total user: cnt


In [344]:
span_order_feature=get_order_feature(span_order_df)
span_loan_feature=get_loan_feature(span_loan_df)


(154852, 5)
33766
count total user: cnt
count total user: cnt
count total user: cnt


ValueError: labels ['active_date'] not contained in axis

In [348]:
span_feature=pd.merge(user_df,span_order_feature,on='uid',how='left')
span_feature=pd.merge(span_feature,span_loan_feature,on='uid',how='left')
span_feature.drop(['active_date'],axis=1,inplace=True)

In [353]:
span_feature.shape
span_data=span_feature

In [328]:
train_data['loan_sum']

0        5.501180
1        0.000000
2        0.000000
3        0.000000
4        0.000000
5        0.000000
6        0.000000
7        0.000000
8        0.000000
9        0.000000
10       6.014775
11       3.862595
12       0.000000
13       5.716524
14       0.000000
15       0.000000
16       0.000000
17       0.000000
18       5.722768
19       6.316423
20       4.723017
21       0.000000
22       0.000000
23       0.000000
24       0.000000
25       4.292651
26       0.000000
27       0.000000
28       6.153414
29       0.000000
           ...   
90963    0.000000
90964    0.000000
90965    0.000000
90966    0.000000
90967    0.000000
90968    5.014930
90969    0.000000
90970    0.000000
90971    0.000000
90972    0.000000
90973    0.000000
90974    0.000000
90975    0.000000
90976    0.000000
90977    0.000000
90978    0.000000
90979    5.501180
90980    0.000000
90981    0.000000
90982    0.000000
90983    0.000000
90984    5.657311
90985    0.000000
90986    0.000000
90987    0

In [329]:
import xgboost as xgb

D:\Anaconda\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [354]:
used_features=list(train_data.columns)
used_features.remove('uid')
used_features.remove('loan_sum')
train_df=train_data[used_features]
train_target=train_data.loan_sum
span_df=span_data[used_features]

In [359]:
dtrain=xgb.DMatrix(train_df,label=train_target)
dspan=xgb.DMatrix(span_df)
our_params={'eta':0.1,'seed':0,'subsample':0.8,'colsample_bytree':0.8,'objective':'reg:linear','max_depth':3,'min_child_weight':1}
model=xgb.train(our_params,dtrain)
train_predict=model.predict(dtrain)
print calc_RMSE(list(train_target),list(train_predict))
span_predict=model.predict(dspan)
span_result=pd.DataFrame()
span_result['uid']=span_data['uid']
span_result['predict']=span_predict

1.90695926505


In [363]:
span_result['predict']

0.3461951

In [364]:
span_result.to_csv('./result.csv',index=False)

In [319]:
def calc_RMSE(true_y,predict_y):
    n=len(true_y)
    result=0
    for i in range(n):
        result+=pow((true_y[i]-predict_y[i]),2)
    return math.sqrt(1.0*result/n)

In [316]:
import math